In [1]:
%matplotlib inline
# %matplotlib notebook
import rasterio
from shapely.geometry import box
import geopandas as gpd
from rasterio.mask import mask
import pandas as pd
import numpy as np
import matplotlib

In [2]:
url_imagen = '/home/diego/Proyecto Vivero/data/01_VIVERO/01_imagenes/imagen_mesones_rgb.tif'
shp_puntos = '/home/diego/Proyecto Vivero/data/01_VIVERO/02_puntos_plantas/plantas_mesones_2.shp'
shp_bandejas = '/home/diego/Proyecto Vivero/data/bandejas.shp'
url_salida = "/home/diego/Proyecto Vivero/train"

In [3]:
puntos = gpd.read_file(shp_puntos)
bandejas = gpd.read_file(shp_bandejas)
raster = rasterio.open(url_imagen)
cuentas = pd.DataFrame(columns=['imagen', 'cant_arboles'])

In [91]:
pl = raster.index(bandejas["geometry"][0].bounds[0],bandejas["geometry"][0].bounds[1])
pt = raster.index(bandejas["geometry"][0].bounds[0],bandejas["geometry"][0].bounds[3])
pr = raster.index(bandejas["geometry"][0].bounds[2],bandejas["geometry"][0].bounds[3])
width = pr[1]-pt[1]
height = pl[0]-pt[0]
print(width, height, pt, pl)

55 61 (1109, 1679) (1170, 1679)


In [7]:
%%time
contador = 0
idx = 0
while (1):
    #transformacion de la imagen
    out_img, out_transform = mask(raster, shapes=[bandejas["geometry"][contador]], crop=True)
    matplotlib.image.imsave('{}/salida{}.png'.format(url_salida,contador), out_img.T)


#interseccion ptos
    cantidad=puntos.intersects(bandejas["geometry"][contador]).value_counts()
    if (True in cantidad.index):
        numero = cantidad[True]
    else:
        numero = 0
    cuentas.loc[contador] = ["bandeja"+str(contador)+".png",numero]

    contador = contador + 1   

KeyError: 515

In [9]:
cuentas.to_csv(url_salida+"/cuentas.csv")